In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os
import pandas as pd

Using TensorFlow backend.


In [2]:
model = load_model('HandEmo.h5')

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis



In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))

    return max(pred_probab), pred_class

def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x,y,w,h = 300, 50, 350, 350
while(cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
    res = cv2.bitwise_and(img, img , mask=mask2)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    median = cv2.GaussianBlur(gray, (5,5), 0)

    kernel_square = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(median, kernel_square, iterations=2)
    opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
    ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

    thresh = thresh[y:y+h , x:x+w]
    contours = cv2.findContours(thresh.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour)>2500:
            x, y, w1, h1= cv2.boundingRect(contour)
            newImage = thresh[y:y+h1 , x:x+w1]
            newImage = cv2.resize(newImage, (50,50))
            pred_probab, pred_class = keras_predict(model, newImage)
            print(pred_class, pred_probab)
            img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

    x,y,w,h = 300, 50, 350,350
    cv2.imshow('Frame',img)
    cv2.imshow('Contours',thresh)
    k = cv2.waitKey(10)
    if k==27:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
1 0.97430134
11 0.49052113
11 0.49052113
3 0.6662943
3 0.6662943
11 0.2953927
11 0.2953927
11 0.6884599
11 0.6884599
2 0.9835554
2 0.9835554
5 0.75804317
4 0.6661737
4 0.83536613
4 0.83536613
4 0.73235935
4 0.73235935
4 0.9652829
4 0.9028745
4 0.9028745
4 0.9409913
4 0.98738354
4 0.9503571
4 0.9107754
4 0.9107754
4 0.8241489
4 0.9438582
4 0.9438582
7 0.7588506
7 0.99987924
7 0.99987924
7 0.8621886
7 0.9847439
7 0.9847439
7 0.7676345
7 0.6309777
7 0.99763966
7 0.85961545
7 0.95903736
7 0.95903736
7 0.99978346
7 0.99978346
7 0.9983456
7 0.9992823
7 0.9974432
7 0.9974432
2 0.28514367
2 0.28514367
7 0.93677276
7 0.93677276
7 0.99301964
7 0.99301964
7 0.42652872
7 0.9986726
7 0.9986726
7 0.99427235
7 0.99427235
7 0.9999423
7 0.9999423
7 0.99979895
7 0.99979895
7 0.9998093
7 0.9998093
7 0.9983816
7 0.9209715
7 0.9209715
7 0.33972448
7 0.99826956
7 0.99826956
7 0.9996131
7 0.9996131
7 0.63238114
7 0.63238114
7 0.9998386
5 0.7517012
5 0.7517012
7 0.9985312
7 0.9985

5 0.8235302
5 0.8697002
5 0.76915014
5 0.7610818
4 0.42477286
4 0.42477286
2 0.256882
2 0.256882
5 0.8318241
5 0.8933769
5 0.96361536
5 0.96361536
5 0.9805772
5 0.9883237
5 0.9922805
5 0.9922805
5 0.9961797
5 0.9961797
5 0.99890697
5 0.99940896
5 0.99940896
5 0.9993493
5 0.99932075
5 0.99932075
5 0.9991035
5 0.9991035
5 0.9992773
5 0.9992773
5 0.9994392
5 0.9993845
5 0.9993845
5 0.9994837
5 0.9996706
5 0.9996706
5 0.99976
5 0.99877244
5 0.9996239
5 0.9996239
5 0.9997521
5 0.99970645
5 0.99970645
5 0.99971706
5 0.9997086
5 0.9997166
5 0.9997166
5 0.99969494
5 0.99901676
5 0.99901676
1 0.92607075
1 0.9180654
1 0.85801655
1 0.85801655
1 0.54429924
1 0.88090324
1 0.8514489
1 0.8514489
1 0.9513207
1 0.84067017
1 0.84764385
1 0.84764385
1 0.8432271
1 0.939766
1 0.939766
1 0.8768201
1 0.94556946
1 0.94157463
1 0.9201539
1 0.9201539
1 0.91646945
1 0.948085
1 0.9604928
1 0.91044074
1 0.91044074
1 0.71290165
11 0.8700558
11 0.8700558
11 0.6949936
11 0.61775184
11 0.8809296
11 0.93745774
11 0.937

9 0.84525585
9 0.84525585
9 0.7956847
11 0.7107651
11 0.7107651
1 0.644518
1 0.644518
9 0.5502324
11 0.80082345
11 0.80082345
9 0.44656327
9 0.44656327
9 0.59243035
9 0.8156179
9 0.8156179
11 0.7740388
11 0.7740388
1 0.5084645
11 0.7326936
1 0.7092493
9 0.5947479
9 0.5947479
11 0.8333465
9 0.59257543
9 0.5696174
11 0.58803695
11 0.8555689
11 0.53623855
11 0.53623855
9 0.8795642
9 0.8795642
11 0.8290535
11 0.47568005
11 0.7050396
11 0.7050396
11 0.45644078
3 0.41170573
11 0.49300244
11 0.49300244
9 0.80089587
11 0.45416674
11 0.6509298
9 0.50309795
11 0.7825332
11 0.7825332
1 0.4523915
1 0.4523915
11 0.8220812
9 0.4905801
9 0.4905801
11 0.84952205
11 0.90172017
1 0.6529669
11 0.55564415
11 0.69596046
11 0.48606393
11 0.5654428
11 0.5654428
4 0.28236905
4 0.28236905
11 0.6790529
11 0.6790529
11 0.84445703
3 0.64246905
11 0.5915595
11 0.5915595
3 0.54084706
3 0.54084706
3 0.62515354
3 0.62515354
11 0.59271103
11 0.59271103
3 0.49038005
3 0.49038005
11 0.52126706
11 0.52126706
3 0.46876198